In [1]:

%load_ext sparkmagic.magics

#Livy endpoint http://10.0.123.114:8998


In [2]:
%manage_spark

MagicsControllerWidget(children=(Tab(children=(ManageSessionWidget(children=(HTML(value='<br/>'), HTML(value='…

In [3]:
%spark?

Docstring:
::

  %spark [-c CONTEXT] [-s SESSION] [-o OUTPUT] [-q [QUIET]]
             [-m SAMPLEMETHOD] [-n MAXROWS] [-r SAMPLEFRACTION] [-u URL]
             [-a USER] [-p PASSWORD] [-t AUTH] [-l LANGUAGE] [-k [SKIP]]
             [-i ID] [-e COERCE]
             [command [command ...]]

Magic to execute spark remotely.

This magic allows you to create a Livy Scala or Python session against a Livy endpoint. Every session can
be used to execute either Spark code or SparkSQL code by executing against the SQL context in the session.
When the SQL context is used, the result will be a Pandas dataframe of a sample of the results.

If invoked with no subcommand, the cell will be executed against the specified session.

Subcommands
-----------
info
    Display the available Livy sessions and other configurations for sessions.
add
    Add a Livy session given a session name (-s), language (-l), and endpoint credentials.
    The -k argument, if present, will skip adding this session if it alr

In [5]:
%spark 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
%%spark
numbers = sc.parallelize([1, 2, 3, 4])
print('First element of numbers is {} and its description is:\n{}'.format(numbers.first(), numbers.toDebugString()))
print('Count of numbers is {} \n{}'.format(numbers.count(), numbers.toDebugString()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

First element of numbers is 1 and its description is:
b'(2) ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274 []'
Count of numbers is 4 
b'(2) ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274 []'

In [7]:
%spark info

Info for running Spark:
    Sessions:
        Name: spark-session	Session id: 3	YARN id: application_1628875993215_0027	Kind: pyspark	State: idle
	Spark UI: http://ip-10-0-123-114.us-west-2.compute.internal:20888/proxy/application_1628875993215_0027/
	Driver Log: http://ip-10-0-113-154.us-west-2.compute.internal:8042/node/containerlogs/container_1628875993215_0027_01_000001/dominospark
    Session configs:
        {'driverMemory': '1000M', 'executorCores': 2}



In [8]:
%%spark -s spark-session
##Only if you have multiple spark sessions or else simple %%spark works
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, IntegerType
import random

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
%%spark -s spark-session
##Only if you have multiple spark sessions or else simple %%spark works

def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1

columns = StructType([ StructField("name", StringType(), True),
                      StructField("value", DoubleType(), True)
                    ])

count = sc.parallelize(range(0, 1000),1) \
             .filter(inside).count()
data = [("Pi",4.0 * count/1000)]

df = spark.createDataFrame(data=data, schema=columns)

df.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|name|value|
+----+-----+
|  Pi|3.096|
+----+-----+

In [11]:
%%spark
hdfs_path='/user/dominospark/large-data-livy-2'
data = []
for i in range(1000):
    data.append((i,random.randint(1,100),random.randint(1,100),random.randint(1,100)))

columns = StructType([ StructField("id", IntegerType(), True),
                       StructField("v1", IntegerType(), True),
                       StructField("v2", IntegerType(), True),
                       StructField("v3", IntegerType(), True) ])

df = spark.createDataFrame(data=data, schema=columns)
df.write.csv(hdfs_path)

df_load = spark.read.csv( hdfs_path, schema=columns)
df_load.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---+---+---+
| id| v1| v2| v3|
+---+---+---+---+
|500|  2| 41| 37|
|501| 59| 45| 92|
|502| 45| 21| 57|
|503| 81| 65| 86|
|504| 14|  3| 98|
|505| 22| 78|  9|
|506| 69| 15| 29|
|507| 95| 24| 10|
|508| 53| 50| 28|
|509| 84| 69| 78|
|510| 89| 26| 12|
|511| 88| 19|  6|
|512| 96| 87| 16|
|513| 73| 51| 95|
|514| 90|  2| 64|
|515| 67| 27| 94|
|516| 96| 47| 85|
|517| 33| 11| 46|
|518| 17| 48| 85|
|519| 87| 53| 32|
+---+---+---+---+
only showing top 20 rows

In [12]:
%spark logs

stdout: 

stderr: 
21/08/17 01:25:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/08/17 01:25:14 INFO RMProxy: Connecting to ResourceManager at ip-10-0-123-114.us-west-2.compute.internal/10.0.123.114:8032
21/08/17 01:25:14 INFO Client: Requesting a new application from cluster with 2 NodeManagers
21/08/17 01:25:15 INFO Configuration: resource-types.xml not found
21/08/17 01:25:15 INFO ResourceUtils: Unable to find 'resource-types.xml'.
21/08/17 01:25:15 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
21/08/17 01:25:15 INFO Client: Will allocate AM container, with 1384 MB memory including 384 MB overhead
21/08/17 01:25:15 INFO Client: Setting up container launch context for our AM
21/08/17 01:25:15 INFO Client: Setting up the launch environment for our AM container
21/08/17 01:25:15 INFO Client: Preparing resources fo

In [13]:

%manage_spark

MagicsControllerWidget(children=(Tab(children=(ManageSessionWidget(children=(HTML(value='<br/>'), HBox(childre…